In [4]:
import pandas as pd
import ast
import pytz
import os
from datetime import datetime
from IPython.display import display

0. Prereqs:
-  set up local dataset directory
- modify local_base_path in config.yaml for dev purpose (do not push this change)

In [5]:
gapower = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/s3/ga/layout_4/per_zipcode_Georgia Power.csv')

1.An example on metrics of interest on one zipcode:

In [16]:
df = gapower[gapower['name'] == '31408 (SAVANNAH)']
# Convert 'timestamp' column to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m-%d-%Y %H:%M:%S')

# Extract year, month, day, hour from 'timestamp'
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute

# TODO: implement 
# df['delta_timestamp'] = (df['timestamp'].diff().dt.total_seconds() / 60).round(0).fillna(0)
# df['delta_percent_cust_a'] = df['percent_cust_a'].diff().fillna(0)

df['outage_freq_x_cust_a'] = df['cust_a'] * df['n_out']
df['duration_weight'] = 15
df['cust_a_x_duration'] = df['cust_a'] * df['duration_weight']
df

/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_59935/3312273585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m-%d-%Y %H:%M:%S')
/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_59935/3312273585.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['timestamp'].dt.year
/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_59935/3312273585.py:7: SettingWithCopyWarning: 
A value is trying to be set on a c

,key,name,cust_a,cust_s,percent_cust_a,etr,etr_confidence,areaId,n_out,timestamp,EMC,year,month,day,hour,minute,outage_freq_x_cust_a,duration_weight,cust_a_x_duration
0,Zip (City),31408 (SAVANNAH),6,6145,0.10,2023-03-16T01:45:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),1,2023-03-16 01:17:50,Georgia Power,2023,3,16,1,17,6,15,90
745,Zip (City),31408 (SAVANNAH),7,6145,0.11,2023-03-17T21:30:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),2,2023-03-17 19:32:52,Georgia Power,2023,3,17,19,32,14,15,105
782,Zip (City),31408 (SAVANNAH),7,6145,0.11,2023-03-17T21:30:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),2,2023-03-17 20:32:52,Georgia Power,2023,3,17,20,32,14,15,105
1552,Zip (City),31408 (SAVANNAH),4,6146,0.02,2023-03-21T12:45:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),1,2023-03-21 11:32:46,Georgia Power,2023,3,21,11,32,4,15,60
6117,Zip (City),31408 (SAVANNAH),4,6145,0.02,2023-03-27T19:30:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),1,2023-03-27 17:02:35,Georgia Power,2023,3,27,17,2,4,15,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976324,Zip (City),31408 (SAVANNAH),4,6182,0.06,2024-02-01T01:00:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),1,2024-01-31 22:48:16,Georgia Power,2024,1,31,22,48,4,15,60
977927,Zip (City),31408 (SAVANNAH),4,6182,0.02,2024-02-02T18:30:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),1,2024-02-02 16:33:17,Georgia Power,2024,2,2,16,33,4,15,60
977985,Zip (City),31408 (SAVANNAH),4,6182,0.02,2024-02-02T18:30:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),1,2024-02-02 16:48:10,Georgia Power,2024,2,2,16,48,4,15,60
978045,Zip (City),31408 (SAVANNAH),4,6182,0.02,2024-02-02T18:30:00Z,LOW,REAG|31408 (SAVANNAH)|Zip (City),1,2024-02-02 17:48:05,Georgia Power,2024,2,2,17,48,4,15,60


2. Formulate into pipeline class to aggregate raw records:

In [9]:
import pandas as pd
import ast
import pytz
import os
import json
import yaml
import glob
from dateutil import tz
from datetime import datetime

class BasePipeline:
    def __init__(self, config, base_file_path):
        self.config = config
        self.base_file_path = base_file_path
        self.geomap = {}
        self._data = pd.DataFrame({})
    
    def construct_file_path(self):
        file_prefix = 'per_outage' if self.config['type'] == 'o' else 'per_county'
        file_path = f"{self.base_file_path}/{self.config['state']}/layout_{self.config['layout']}/{file_prefix}_{self.config['name']}.csv"
        return file_path.replace('//', '/')

    def load_data(self):
        try:
            file_path = self.construct_file_path()
            print(file_path)
            self._data = pd.read_csv(file_path)
            # with open('zip_to_county_name.json', 'r') as json_file:
            #     self.geomap['zip_to_county_name'] = json.load(json_file)
            # with open('zip_to_county_fips.json', 'r') as json_file:
            #     self.geomap['zip_to_county_fips'] = json.load(json_file)
        except Exception as e:
            print(f"An error occurred during file loading: {e}")
            
    def transform(self):
        raise NotImplementedError

    def standardize(self):
        """
        Generic method to compute and output standardized metrics
        """
        self.load_data()
        self.transform()
        grouped = self._data.groupby('outage_id').apply(self._compute_metrics).reset_index().round(2)
        self._data = pd.merge(grouped, self._data, on=['outage_id', 'timestamp'], how='inner')
        
        self._data['state'] = self.config['state']
        if self.config['state'] != 'ca':
            self._data['utility_provider'] = self.config['name'] 
            self._data['county'] = self._data['zipcode'].map(self.geomap) 
        
        self._data = self._data[[
            'utility_provider', 'state', 'county', 'zipcode',
            'outage_id', 'start_time', 'end_time', 'lat', 'lng', 
            'duration', 'duration_max', 'duration_mean', 'customer_affected_mean', 'total_customer_outage_time', 'total_customer_outage_time_max', 'total_customer_outage_time_mean'
        ]]
        
        return self._data
    
    def get_dataframe(self):
        return self._data
    
    def _compute_metrics(self, group):
        """
        Generic method to compute standardized metrics, used for being apply in DataFrame.groupby method, 
        given dataframe being transformed with standardized column names
        """
        duration = (group['end_time'] - group['start_time']).dt.total_seconds() / 60
        duration_max = duration + 15
        duration_mean = (duration + duration_max) / 2
        customer_affected_mean = group['customer_affected'].mean()
        
        total_customer_outage_time = 15 * (group['customer_affected'].sum() - group['customer_affected'].iloc[0]) + (group['timestamp'].iloc[0] - group['start_time'].iloc[0]).total_seconds() / 60 * group['customer_affected'].iloc[0]
        total_customer_outage_time_max = total_customer_outage_time + 15 * group['customer_affected'].iloc[-1]
        total_customer_outage_time_mean = (total_customer_outage_time + total_customer_outage_time_max) / 2

        return pd.Series({
            'timestamp': group['end_time'].iloc[-1],
            'duration': duration.iloc[-1],
            'duration_max': duration_max.iloc[-1],
            'duration_mean': duration_mean.iloc[-1],
            'customer_affected_mean': customer_affected_mean,
            'total_customer_outage_time': total_customer_outage_time,
            'total_customer_outage_time_max': total_customer_outage_time_max,
            'total_customer_outage_time_mean': total_customer_outage_time_mean
        })

In [14]:
class GA4TX5(BasePipeline):
    def transform(self):
        '''
        Method to implement
        '''
        # Convert 'timestamp' column to datetime format
        self._data['timestamp'] = pd.to_datetime(self._data['timestamp'], format='%m-%d-%Y %H:%M:%S')

        # Extract year, month, day, hour from 'timestamp'
        self._data['year'] = self._data['timestamp'].dt.year
        self._data['month'] = self._data['timestamp'].dt.month
        self._data['day'] = self._data['timestamp'].dt.day
        self._data['hour'] = self._data['timestamp'].dt.hour
        # self._data['minute'] = self._data['timestamp'].dt.minute
        
        # TODO: apply rules for getting n_out 
        # df['delta_timestamp'] = (df['timestamp'].diff().dt.total_seconds() / 60).round(0).fillna(0)
        # df['delta_percent_cust_a'] = df['percent_cust_a'].diff().fillna(0)
        
        # element wise metrics computation
        self._data['duration_weight'] = 15
        self._data['outage_freq_x_cust_a'] = self._data['cust_a'] * self._data['n_out']
        self._data['cust_a_x_duration'] = self._data['cust_a'] * self._data['duration_weight']

        self._data['outage_freq_x_cust_a'] = self._data['cust_a'] * self._data['n_out']
        self._data['duration_weight'] = 15
        self._data['cust_a_x_duration'] = self._data['cust_a'] * self._data['duration_weight']
    
    def to_hourly_zip_level(self):
        '''
        Method to implement
        '''
        self.load_data()
        display(self._data)
        self.transform()
        result = self._data.groupby(['name', 'EMC', 'year', 'month', 'day', 'hour']).apply(self._compute_metrics).reset_index()
        
        self._data = result
        # [[
        #     'name', 'timestamp','EMC','year','month','day','hour',
        #     'cust_a','cust_s','percent_cust_a','n_out',
        #     # 'delta_timestamp', 'delta_percent_cust_a', 
        #     'duration_weight', 'cust_a_x_duration', 'outage_freq_x_cust_a'
        # ]]
        
        return self._data
        
    def standardize(self):
        raise NotImplementedError
    
    def _compute_metrics(self, group):
        return pd.Series({
        'customer_affected_mean': group['cust_a'].mean(),
        'cust_served_mean': group['cust_s'].mean(),
        'percent_customer_affected_mean': group['percent_cust_a'].mean(),
        'outage_freq': group['n_out'].max(),
        'duration': group['duration_weight'].sum(),
        'outage_freq_x_cust_a': group['outage_freq_x_cust_a'].sum(),
        'cust_a_x_duration': group['cust_a_x_duration'].sum()
        })

In [15]:
# driver code for testing
with open('/Users/xuanedx1/github/outage-data-scraper/app/pipeline/config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    base_file_path = config['globals']['LOCAL_FILE_BASE_PATH']

# Instantiate a BasePipeline object for each provider in the configuration
for provider in config['providers']:
    pipeline = GA4TX5(provider, base_file_path)
    pipeline.to_hourly_zip_level()
    display(pipeline._data)
    # pipeline._data.to_csv(f"{pipeline.config['name']}_o.csv")

/Users/xuanedx1/github/outage-data-scraper/data/s3/ga/layout_4/per_county_Georgia Power.csv


,key,name,cust_a,cust_s,percent_cust_a,etr,etr_confidence,areaId,n_out,timestamp,EMC
0,county,PAULDING,4,4218,0.02,2023-03-15T21:45:00Z,LOW,REAG|PAULDING|county,1,03-15-2023 20:32:51,Georgia Power
1,county,CLAYTON,4,117715,0.01,2023-03-15T23:30:00Z,LOW,REAG|CLAYTON|county,3,03-15-2023 20:32:51,Georgia Power
2,county,TATTNALL,20,5009,0.40,ETR-NULL,LOW,REAG|TATTNALL|county,2,03-15-2023 20:32:51,Georgia Power
3,county,HENRY,4,52034,0.01,2023-03-15T21:45:00Z,LOW,REAG|HENRY|county,1,03-15-2023 20:32:51,Georgia Power
4,county,TERRELL,4,3232,0.03,ETR-EXP,LOW,REAG|TERRELL|county,1,03-15-2023 20:32:51,Georgia Power
...,...,...,...,...,...,...,...,...,...,...,...
469549,county,BERRIEN,4,4022,0.02,2024-02-03T11:00:00Z,LOW,REAG|BERRIEN|county,1,02-03-2024 03:03:04,Georgia Power
469550,county,DEKALB,178,339050,0.05,2024-02-03T05:30:00Z,LOW,REAG|DEKALB|county,3,02-03-2024 03:03:04,Georgia Power
469551,county,JACKSON,8,10477,0.08,2024-02-15T21:30:00Z,LOW,REAG|JACKSON|county,2,02-03-2024 03:03:04,Georgia Power
469552,county,CHATHAM,4,157824,0.01,2024-02-03T14:00:00Z,LOW,REAG|CHATHAM|county,1,02-03-2024 03:03:04,Georgia Power


,name,EMC,year,month,day,hour,customer_affected_mean,cust_served_mean,percent_customer_affected_mean,outage_freq,duration,outage_freq_x_cust_a,cust_a_x_duration
0,APPLING,Georgia Power,2023,3,16,16,4.0,2988.0,0.03,1.0,30.0,8.0,120.0
1,APPLING,Georgia Power,2023,3,22,21,4.0,2989.0,0.10,2.0,30.0,16.0,120.0
2,APPLING,Georgia Power,2023,3,22,22,4.0,2989.0,0.10,2.0,30.0,16.0,120.0
3,APPLING,Georgia Power,2023,3,27,12,4.0,2998.0,0.03,1.0,15.0,4.0,60.0
4,APPLING,Georgia Power,2023,3,27,13,4.0,2998.0,0.03,1.0,15.0,4.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176826,WORTH,Georgia Power,2024,1,24,14,4.0,1390.0,0.07,1.0,15.0,4.0,60.0
176827,WORTH,Georgia Power,2024,1,24,15,4.0,1390.0,0.07,1.0,60.0,16.0,240.0
176828,WORTH,Georgia Power,2024,1,24,16,4.0,1390.0,0.07,1.0,45.0,12.0,180.0
176829,WORTH,Georgia Power,2024,1,24,17,4.0,1390.0,0.07,1.0,60.0,16.0,240.0


Tasks:

1. Formulate pipeline class for GA1TX8 layout by applying the same methodology
2. Implement the rules for getting n_out (number of outage)
3. Fill all non outage hours with values of 0